<a href="https://colab.research.google.com/github/vhovenga/CS4850/blob/main/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers, models
import gspread
import pandas as pd
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials
import pickle 
import pywt
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from keras.utils import to_categorical


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:


histidine_train = gc.open("histidine_image_scan_000_Spec.Data 1_F").sheet1
glycine_train = gc.open("GlycineMap_1sInt_50x50scans_001_Spec.Data 1_F").sheet1
leucine_train = gc.open("Leucine_image_scan_000_Spec.Data 1_F").sheet1
asparagine_train = gc.open("AsparagineMap_1sInt_50x50scans_002_Spec.Data 1_F").sheet1
lysine_train = gc.open("Lysine").sheet1

histidine_test = gc.open("Histidine_Test").sheet1
glycine_test = gc.open("Glycine_Test").sheet1
leucine_test = gc.open("Leucine_Test").sheet1
asparagine_test = gc.open("Asparagin_Test").sheet1
lysine_test = gc.open("Lysine_Test").sheet1


In [ ]:
hist_train = histidine_train.get_all_records()
gly_train = glycine_train.get_all_records()
leu_train = leucine_train.get_all_records()
asp_train = asparagine_train.get_all_records()
lys_train = lysine_train.get_all_records()

hist_test = histidine_test.get_all_records()
gly_test  = glycine_test.get_all_records()
leu_test = leucine_test.get_all_records()
asp_test = asparagine_test.get_all_records()
lys_test = lysine_test.get_all_records()

In [ ]:
hist_train = pd.DataFrame(hist_train)
gly_train = pd.DataFrame(gly_train)
leu_train = pd.DataFrame(leu_train)
asp_train = pd.DataFrame(asp_train)
lys_train = pd.DataFrame(lys_train)

hist_test = pd.DataFrame(hist_test)
gly_test = pd.DataFrame(gly_test)
leu_test = pd.DataFrame(leu_test)
asp_test = pd.DataFrame(asp_test)
lys_test = pd.DataFrame(lys_test)

In [ ]:
# Remove unnecessary rows and columns
hist_train = hist_train.drop(index=0,columns="X-Axis")
gly_train = gly_train.drop(index=0,columns="X-Axis")
leu_train = leu_train.drop(index=0,columns="X-Axis")
asp_train = asp_train.drop(index=0,columns="X-Axis")
lys_train = lys_train.drop(index=0,columns="X-Axis")

hist_test = hist_test.drop(index=0,columns="X-Axis")
gly_test = gly_test.drop(index=0,columns="X-Axis")
leu_test = leu_test.drop(index=0,columns="X-Axis")
asp_test = asp_test.drop(index=0,columns="X-Axis")
lys_test = lys_test.drop(index=0,columns="X-Axis")

In [ ]:
# Create labels for the data
hist_train_lab = [2]* hist_train.shape[1]
gly_train_lab = [1]* gly_train.shape[1]
leu_train_lab = [3]* leu_train.shape[1]
asp_train_lab = [0]* asp_train.shape[1]
lys_train_lab = [4]* lys_train.shape[1]

hist_test_lab = [2]* hist_test.shape[1]
gly_test_lab = [1]* gly_test.shape[1]
leu_test_lab = [3]* leu_test.shape[1]
asp_test_lab = [0]* asp_test.shape[1]
lys_test_lab = [4]* lys_test.shape[1]


In [ ]:
hist_train = hist_train.T
gly_train = gly_train.T
leu_train = leu_train.T
asp_train = asp_train.T
lys_train = lys_train.T

hist_test = hist_test.T
gly_test = gly_test.T
leu_test = leu_test.T
asp_test = asp_test.T
lys_test = lys_test.T

In [ ]:
# Add labels to data
hist_train['Label'] = hist_train_lab
gly_train['Label'] = gly_train_lab
leu_train['Label'] = leu_train_lab
asp_train['Label'] = asp_train_lab
lys_train['Label'] = lys_train_lab 

hist_test['Label'] = hist_test_lab
gly_test['Label'] = gly_test_lab 
leu_test['Label'] = leu_test_lab 
asp_test['Label'] = asp_test_lab 
lys_test['Label'] = lys_test_lab 


In [ ]:
# Concatinate all data into one dataframe and then shuffle the rows. 
train = pd.concat([hist_train, gly_train, leu_train, asp_train, lys_train])
test = pd.concat([hist_test, gly_test, leu_test, asp_test, lys_test])
data = pd.concat([test, train])

data = data.sample(frac=1)

In [ ]:
# 
data_lab = data.pop("Label")

data_lab = np.reshape(np.array(data_lab),(len(data_lab),1))

data_inpt = np.array(data)


NameError: ignored

In [ ]:
data = (data_inpt, data_lab)

out_data = open("/content/drive/My Drive/Colab Notebooks/data.pickle","wb")
pickle.dump(data, out_data)
out_data.close()


In [ ]:
in_data =open("/content/drive/My Drive/Colab Notebooks/data.pickle", "rb")
data = pickle.load(in_data)
in_data.close()


In [ ]:
n_feature_maps = 64
nb_classes = 5
reg_rate = 0


input_layer = keras.layers.Input([data[0].shape[1],1])

# BLOCK 1

conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, kernel_regularizer=regularizers.l2(reg_rate), padding='same')(input_layer)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, kernel_regularizer=regularizers.l2(reg_rate), padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, kernel_regularizer=regularizers.l2(reg_rate),padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_1 = keras.layers.add([shortcut_y, conv_z])
output_block_1 = keras.layers.Activation('relu')(output_block_1)
output_block_1 = keras.layers.Dropout(.1)(output_block_1)

# BLOCK 2

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8,kernel_regularizer=regularizers.l2(reg_rate),padding='same')(output_block_1)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5,kernel_regularizer=regularizers.l2(reg_rate), padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, kernel_regularizer=regularizers.l2(reg_rate),padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_2 = keras.layers.add([shortcut_y, conv_z])
output_block_2 = keras.layers.Activation('relu')(output_block_2)
output_block_2 = keras.layers.Dropout(.1)(output_block_2)
# BLOCK 3

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, kernel_regularizer=regularizers.l2(reg_rate), padding='same')(output_block_2)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, kernel_regularizer=regularizers.l2(reg_rate),padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, kernel_regularizer=regularizers.l2(reg_rate),padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# no need to expand channels because they are equal
shortcut_y = keras.layers.BatchNormalization()(output_block_2)

output_block_3 = keras.layers.add([shortcut_y, conv_z])
output_block_3 = keras.layers.Activation('relu')(output_block_3)

# FINAL

gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

output_layer = keras.layers.Dense(128, activation='relu')(gap_layer)
output_layer = keras.layers.Dropout(.2)(output_layer)
output_layer = keras.layers.Dense(nb_classes, activation='softmax')(output_layer)

res_net = keras.models.Model(inputs=input_layer, outputs=output_layer)

res_net.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=.00001),
             metrics=['accuracy'])


In [ ]:
# Data preprocessing 

inpt = data[0].reshape(data[0].shape[0],data[0].shape[1],1)
inpt = inpt.astype('float32')

lab = data[1]
lab = lab.astype('float32')
lab = to_categorical(lab)


# Discrete wavelet transformation using Haar wavelets.
# x_train = pywt.dwt(x_train,'haar')[0]
# x_test = pywt.dwt(x_test,'haar')[0]


In [ ]:
#model = keras.Sequential()
#model.add(keras.layers.experimental.preprocessing.Normalization())
#model.add(res_net)

#model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(),
            # metrics=['accuracy'])


trained = res_net.fit(inpt, lab, epochs = 25, batch_size=64,validation_split=.2)

Epoch 1/25
163/163 [==============================] - 21s 131ms/step - loss: 1.4516 - accuracy: 0.3888 - val_loss: 1.6090 - val_accuracy: 0.1827
Epoch 2/25
163/163 [==============================] - 21s 129ms/step - loss: 1.1145 - accuracy: 0.6100 - val_loss: 1.3715 - val_accuracy: 0.3181
Epoch 3/25
163/163 [==============================] - 21s 130ms/step - loss: 0.8792 - accuracy: 0.7720 - val_loss: 1.0105 - val_accuracy: 0.6492
Epoch 4/25
163/163 [==============================] - 21s 131ms/step - loss: 0.6295 - accuracy: 0.8695 - val_loss: 0.6151 - val_accuracy: 0.8800
Epoch 5/25
163/163 [==============================] - 21s 132ms/step - loss: 0.4757 - accuracy: 0.8982 - val_loss: 0.3964 - val_accuracy: 0.9542
Epoch 6/25
163/163 [==============================] - 22s 132ms/step - loss: 0.3651 - accuracy: 0.9314 - val_loss: 0.2772 - val_accuracy: 0.9842
Epoch 7/25
163/163 [==============================] - 22s 133ms/step - loss: 0.2737 - accuracy: 0.9668 - val_loss: 0.1893 - val_ac

In [ ]:
res_net.save('/content/drive/MyDrive/Colab Notebooks/trained_class')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/trained_class/assets


In [ ]:
plt.plot(epoch_vec, accuracy_vec, label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

plt.savefig("training.png")

NameError: ignored